# Fase 1: Extracción de Datos de API de MoviesDataset

API: https://rapidapi.com/SAdrian/api/moviesdatabase

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

## API Keys:

- API_key Sharon = '2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1'
- API_key3 Silvia = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
- API_key3 Fer  = 'd215e5295fmsh4ea493c17eb16a6p1ad050jsnd6897569237c'
- API_key Isa1  = '7a833b5f94msh046ef16d4b5eb88p18a187jsn006b25582aff'
- API_key Isa2 = '555502744dmsh2f20b5fd2ccbb41p11ac03jsndaa25d7ef539'
- API_key Nuria1 = '0d34f7de1bmsh8ff8ef70a80e175p156753jsn5ea1edd82423'
- API_key Nuria2 = '505782c7fbmsh5a469e43b5fecf5p138dfajsna7d8efc4677f''



In [2]:
#creamos el diccionario donde alamcenaremos la información
dic_API_Pelis = {'Tipo':[],
                 'Nombre':[],
                 'Año_estreno':[],
                 'Mes_estreno': [],
                 'Id':[],
                 'Genero':[]}

#Creamos varias listas para guardar la información de cada key del diccionario que nos devuelve la función. 
lista_tipos = []
lista_nombre = []
lista_anno = []
lista_mes = []
lista_id = []
lista_generos = []

#creamos las variables relacionadas con el ENDPOINT
url = "https://moviesdatabase.p.rapidapi.com/titles"

# creamos una lista para almacenar las respuestas de la funcion de llamada a la API por si las queremos consultar
lista_respuestas = []
lista_info_respuestas = []

# creamos una lista con los tipos que queremos buscar
titleType_list = ['movie', 'short']


In [3]:
# Definimos una fucnción para extraer información de la API y almacenarla en un diccionario

def llamar_API (url,type,year,genere,**dict):
    '''
    Esta función realiza una llamada a una API a partir de una url y guarda la información en un diccionario.

    Parametros:
    -----------
    url (str): la dirección url de la API a la que queremos hacer la llamada
    type (str): El tipo de pelícua que queremos buscar. 
    year (str): El año de la película que queremos buscar.
    genere (dtr): El género que ueremos buscar. 
    dict (dict): el diccionario dónde se almacenará la información

    Returns:
    -------
    dict (dict): el diccionario donde se almacenará la información
    '''
    #definimos la querystring 
    querystring = {"titleType":type,"year":year,"genre":genere}

    #definimos una lista con todas las API__key
    API_key_list = ['2d7083a699mshd0e8039e2063bedp140338jsn9b141653fce1',
                    '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2', 
                    'd215e5295fmsh4ea493c17eb16a6p1ad050jsnd6897569237c', 
                    '7a833b5f94msh046ef16d4b5eb88p18a187jsn006b25582aff',
                    '555502744dmsh2f20b5fd2ccbb41p11ac03jsndaa25d7ef539',
                    '0d34f7de1bmsh8ff8ef70a80e175p156753jsn5ea1edd82423',
                    '505782c7fbmsh5a469e43b5fecf5p138dfajsna7d8efc4677f' ]

    #Asignamos un valor inicial a la API_key
    API_key = API_key_list[0]
        

    #iniciamos un loop while para navegar por las distintas páginas
    while url.startswith('https'):

        #definimos el headers, necesario para hacer la llamada a la API
        headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"} 

        #realizmos la llamada 
        response_pelis = requests.get(url, headers=headers, params=querystring)

        #almacenamos los códigos y la informacion de respuesta por si lo queremos consultar
        lista_respuestas.append(response_pelis.status_code)
        lista_info_respuestas.append(response_pelis.reason)

        #comprobamos si la respuesta a la llamada ha sido exitosa = 200
        if response_pelis.status_code == 200:

            #convertimos los resulatos a formato json
            js_response_pelis = response_pelis.json()

            #guardamos la inforación de la API dónde tenemos la dirección de la siguiente página
            next = js_response_pelis['next']

            #guardamos la parte donde tenemos la infomación en una variable
            info_pelis = js_response_pelis['results']

            #guardamos toda la información en el diccionario con un for loop    
            for peli in info_pelis:

                dict['Genero'].append(genere)
                dict['Tipo'].append(peli['titleType']['text'])
                dict['Nombre'].append(peli['titleText']['text'])
                dict['Id'].append(peli['id'])
                dict['Año_estreno'].append(peli['releaseYear']['year'])

                #Hacemos un try/expect para las películas que no se han estrenado todavía            
                try:
                    dict['Mes_estreno'].append(peli['releaseDate']['month'])
                except:            
                    dict['Mes_estreno'].append('por estrenar')

            #iniciamos el loop try-except para que cree la variable de la nueva url mientras tengamos una url en ´next´
            try:
                #definimos la base de la nueva url
                url_fixed = "https://moviesdatabase.p.rapidapi.com"
                #definimos la nueva url
                url=url_fixed+next
                #definimos el nuevo querystring
                querystring = {}

            except:
                break
        
        #definimos un elif en caso de que la llamada nos diga que hemos hecho demasiadas requests = 429
        elif response_pelis.status_code == 429:

            #comprobamos si el la API key NO es la última de la lista de las API_keys
            if API_key_list.index(API_key) < len(API_key_list)-1:

                #cambiamos la API_key por la próxima en la lista
                i = API_key_list.index(API_key)
                API_key = API_key_list[i+1]

            #si la API key SI es la última en la lista, volvemos a asignarle la primera de la lista
            else:
                API_key = API_key_list[0] 

            #le decimos al código que continue con el while loop    
            continue

               
        #definimos una opción para que nos diga el tipo de error que nos ha dado en el caso de que la respuesta a la llamada no sea ni 200 ni 429.
        else:
            print(f'Error {response_pelis.status_code} en la llamada a la API')
            continue


    #mostramos los resultados
    if response_pelis.status_code == 200:
        print(f'Fin: Se han añadido l@s {type}s del género {genere} a tu diccionario.')

In [4]:
# Definimos el género 
# genere = 'Action'
# genere = 'Drama'
genere = 'Comedy'

#Iteramos por cada año, desde el 2010 hasta el 2025 (para que incluya el 2024).
for year in range(2010,2025):
    print('Año:', year)

    #Iteramos por cada tipo: Shot and Movie
    for type in titleType_list:

        #iniciamos la funcion que llama a la API     
        llamar_API(url,type, year, genere, **dic_API_Pelis)

#Añadimos la información en las listas con un extend
lista_tipos.extend(dic_API_Pelis['Tipo'])
lista_nombre.extend(dic_API_Pelis['Nombre'])
lista_anno.extend(dic_API_Pelis['Año_estreno'])
lista_mes.extend(dic_API_Pelis['Mes_estreno'])
lista_id.extend(dic_API_Pelis['Id'])
lista_generos.extend(dic_API_Pelis['Genero'])

Año: 2010
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.
Año: 2011
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.
Año: 2012
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.
Año: 2013
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.
Año: 2014
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.
Año: 2015
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu diccionario.
Año: 2016
Fin: Se han añadido l@s movies del género Comedy a tu diccionario.
Fin: Se han añadido l@s shorts del género Comedy a tu dicci

## Información total obtenida:

* ACTION 2010 to 2024 : 71047, duplicated 25306, clean 45310 
* DRAMA 2010 to 2024 : 293028, duplicated 0, clean 293028
* COMEDY 2010 to 2024 : 155444, duplicated 0, clean 155444

In [ ]:
#Creamos un zip con las listas que hemos creado para que nos guarde la infoamción de cada una de las pelis en una tupla
# y guardamos las tuplas en una lista
zip_fase1 = zip(lista_tipos, lista_nombre, lista_anno, lista_mes, lista_id, lista_generos)
lista_tuplas_fase1 = list(zip_fase1)
len(lista_tuplas_fase1)

In [7]:
# Convertimos la lista de tuplas con la informacion a Data Frame
df_fase1=pd.DataFrame(lista_tuplas_fase1)
df_fase1.columns = ['tipo_pelicula', 'titulo_pelicula', 'anno_estreno', 'mes_estreno', 'id_pelicula','genero_pelicula']


In [8]:
# Visualozamos la información que tenemos
df_fase1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155444 entries, 0 to 155443
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   tipo_pelicula    155444 non-null  object
 1   titulo_pelicula  155444 non-null  object
 2   anno_estreno     155444 non-null  int64 
 3   mes_estreno      146921 non-null  object
 4   id_pelicula      155444 non-null  object
 5   genero_pelicula  155444 non-null  object
dtypes: int64(1), object(5)
memory usage: 7.1+ MB


In [9]:
# Visualizamos la suma de valores nulos que tenemos
df_fase1.duplicated().sum()

0

In [10]:
#Borramos los duplicados y nos aseguramos que si hay elementos sin valor le aplicamos N/A
df_fase1_final = df_fase1.drop_duplicates(subset='id_pelicula')
df_fase1_final.fillna('N/A', inplace=True)

In [11]:
# Volvemos a visualizar la información que tenemos ya limpia
df_fase1_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155444 entries, 0 to 155443
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   tipo_pelicula    155444 non-null  object
 1   titulo_pelicula  155444 non-null  object
 2   anno_estreno     155444 non-null  int64 
 3   mes_estreno      155444 non-null  object
 4   id_pelicula      155444 non-null  object
 5   genero_pelicula  155444 non-null  object
dtypes: int64(1), object(5)
memory usage: 8.3+ MB


In [12]:
# Convertimos el DataFrame a formato .csv
ruta = f'https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase1_csv/Fase1_2010_to_2024_{genere}.csv'
df_fase1_final.to_csv(ruta, index=False)